In [1]:
import pandas as pd
import xlrd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# read file
attacks = pd.read_csv(r'C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\attacks.csv', encoding = 'latin-1')

In [3]:
# explore dataset
attacks.head(500)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


In [4]:
attacks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [5]:
# clean columns' names
attacks.columns = attacks.columns.str.lower().str.strip()
attacks.columns

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal (y/n)', 'time',
       'species', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number.1', 'case number.2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

In [6]:
# drop irrelevant columns
clean_attacks = attacks.drop(['unnamed: 22', 'unnamed: 23'], axis = 1)

In [7]:
# drop lines if more than 22 columns are null
allnull = attacks.isna().sum(axis = 1) >= 22
notnull = ~allnull
clean_attacks = attacks[notnull]

In [8]:
# clean 'fatal' column
import re
clean_attacks.rename(columns = {'fatal (y/n)':'fatal'}, inplace = True)
clean_attacks['fatal'] = clean_attacks['fatal'].str.lower().str.strip().replace('2017','u', regex = True).replace('m','u', regex = True).replace('unknown','u', regex = True)
clean_attacks['fatal'].fillna('u', inplace = True)

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\3256031532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks.rename(columns = {'fatal (y/n)':'fatal'}, inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\3256031532.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['fatal'] = clean_attacks['fatal'].str.lower().str.strip().replace('2017','u', regex = True).replace('m','u', regex = True).replace('unknown','u', regex = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\3256031532.py:5: SettingWithCopyWarning: 

In [9]:
# 'fatal' column analysis
def percentage(x):
    return round((100 * x)/6302, 2)
fatal_df = pd.DataFrame(clean_attacks['fatal'].value_counts())
fatal_df.loc[:,'%'] = fatal_df.apply(percentage, axis=0)
fatal_df

,fatal,%
n,4301,68.25
y,1389,22.04
u,612,9.71


In [10]:
# clean 'sex' column
clean_attacks2 = clean_attacks['sex'].fillna('u', inplace = True)
clean_attacks2 = clean_attacks['sex'].str.lower().str.strip().replace('.', 'u').replace('lli', 'u').replace('n', 'u')
clean_attacks['sex'] = clean_attacks2

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\1072108464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks2 = clean_attacks['sex'].fillna('u', inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\1072108464.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['sex'] = clean_attacks2


In [11]:
# 'sex' column analysis
clean_attacks2.value_counts()
sex_df = pd.DataFrame(clean_attacks2.value_counts())
sex_df.loc[:,'%'] = sex_df.apply(percentage, axis=0)
sex_df

,sex,%
m,5096,80.86
f,637,10.11
u,569,9.03


In [12]:
# 'fatal' X 'sex' analysis
fatal_sex = clean_attacks[['fatal', 'sex']].groupby('fatal').value_counts()
fatal_sex_df = pd.DataFrame(fatal_sex) 
fatal_sex_df.loc[:,'%'] = fatal_sex_df.apply(percentage, axis=0)
fatal_sex_df

0      %
fatal sex             
n     m    3482  55.25
      f     468   7.43
      u     351   5.57
u     m     441   7.00
      u     109   1.73
      f      62   0.98
y     m    1173  18.61
      u     109   1.73
      f     107   1.70

In [13]:
clean_attacks['age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
  

In [14]:
import re
clean_attacks['age'] = clean_attacks['age'].str.lower().str.strip().fillna('u')
clean_attacks['age'] = clean_attacks['age'].str.replace(r'x|a.m.|f|make line green|teens|teen|"young"|young|adult|\(adult\)|elderly|"middle-age"|>50', 'u')
clean_attacks['age'] = clean_attacks['age'].str.replace('20s','25').str.replace('30s','35').str.replace('40s','45').str.replace('50s','55').str.replace("60's","65").str.replace("60s","65").str.replace('mid-25','25').str.replace('mid-35','35')
clean_attacks['age'] = clean_attacks['age'].str.replace('ca.', '').str.replace('both 11', '11 & 11').str.replace('2 to 3 months', '0.3').str.replace('18 months', '1.5').str.replace('9 months', '0.7')

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\841365645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['age'] = clean_attacks['age'].str.lower().str.strip().fillna('u')
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\841365645.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_attacks['age'] = clean_attacks['age'].str.replace(r'x|a.m.|f|make line green|teens|teen|"young"|young|adult|\(adult\)|elderly|"middle-age"|>50', 'u')
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\841365645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [15]:
def clean_numbers(x):
    if 'or' in x or 'to' in x:
        n1 = re.findall('^\d+', x)
        n2 = re.findall('\d+$', x)
        lista_n = n1 + n2
        lista_int = list(map(int, lista_n)) 
        y = sum(lista_int)/2
        return(y)
    elif '½' in x:
        z = re.sub('½', '.5', x)
        return(float(z))
    else:
        return(x)

clean_attacks['age'] = clean_attacks['age'].apply(clean_numbers) 

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\3520995153.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['age'] = clean_attacks['age'].apply(clean_numbers)


In [16]:
def clean_and(x):
    if type(x) == str and '&' in x:
        return re.split('[&,]', x)
    else:
        return [x]
        
clean_attacks['age'] = clean_attacks['age'].apply(clean_and) 

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\903675643.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['age'] = clean_attacks['age'].apply(clean_and)


In [36]:
new_df = clean_attacks.explode('age')
new_df['age'] = new_df['age'].str.strip().str.replace('?', '')
new_df['age'] = new_df['age'].apply(lambda x: np.nan if x == 'u' or x == '' else float(x))

new_df['age'].size 
# 6332 cases

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_11888\3961293361.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_df['age'] = new_df['age'].str.strip().str.replace('?', '')


6332

In [ ]:
'''
adult = 2
teens = 16
elderly = 1
young = 3 
middle-age = 1 

6302
23: classificações


Ao final: 
    1) tudo o que for ? = n 
    2) tudo o que for n = np.nan
    
Read me: o que o recrutador lê; o que está escrito no markDown

'''
